**# HUMAN SCREAM DETECTION**

In [3]:
!apt-get update
!apt-get install -y libportaudio2
!pip install sounddevice
import sounddevice as sd

Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:5 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]
Get:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Get:11 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ Packages [62.9 kB]
Get:12 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [1,304 kB]
Get:13 http://archive.ubuntu.com/ubuntu jamm

In [4]:
# # Importing the libraries
# import numpy as np
# import librosa
# from sklearn.model_selection import train_test_split
# from sklearn.preprocessing import LabelEncoder
# from tensorflow.keras import layers, models, callbacks
# from tensorflow.keras.utils import to_categorical
# import sounddevice as sd
# import matplotlib.pyplot as plt
# import soundfile as sf
# from scipy.io.wavfile import write
# from IPython.display import display, Audio
# Importing the libraries
import numpy as np
import librosa
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras import layers, models, callbacks
from tensorflow.keras.utils import to_categorical
!pip install sounddevice # Install sounddevice library
import sounddevice as sd # Now you should be able to import it
import matplotlib.pyplot as plt
import soundfile as sf
from scipy.io.wavfile import write
from IPython.display import display, Audio

In [5]:
# Function to extract features from audio files using librosa
def extract_features(file_path, mfcc=True, chroma=True, mel=True):
    y, sr = librosa.load(file_path, mono=True)
    features = []
    if mfcc:
        mfccs = np.mean(librosa.feature.mfcc(y=y, sr=sr, n_mfcc=13), axis=1)
        features.extend(mfccs)
    if chroma:
        chroma = np.mean(librosa.feature.chroma_stft(y=y, sr=sr), axis=1)
        features.extend(chroma)
    if mel:
        mel = np.mean(librosa.feature.melspectrogram(y=y, sr=sr), axis=1)
        features.extend(mel)
    return features


In [7]:
# Example to train the model, evaluate, and test with recorded audio
scream_folder = "/content/169810__missozzy__female-scream-01_0.wav"
non_scream_folder = "/content/clnsp104.wav"

In [8]:
# Function to load and preprocess the dataset
def load_and_preprocess_data(scream_folder, non_scream_folder):
    features = []
    labels = []

    # Load scream data
    import os
    for filename in os.listdir(scream_folder):
        path = os.path.join(scream_folder, filename)
        feature = extract_features(path)
        features.append(feature)
        labels.append(1)  # Use integer labels, e.g., 1 for scream

    # Load non-scream data
    for filename in os.listdir(non_scream_folder):
        path = os.path.join(non_scream_folder, filename)
        feature = extract_features(path)
        features.append(feature)
        labels.append(0)  # Use integer labels, e.g., 0 for non-scream

    X = np.array(features)
    y = np.array(labels)

    # Encode labels using LabelEncoder
    le = LabelEncoder()
    y = le.fit_transform(y)

    # Convert to one-hot encoding
    y = to_categorical(y)

    # Split data into training and testing sets
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

    return X_train, X_test, y_train, y_test, le

In [9]:
# Define the model
def create_model(input_shape):
    model = models.Sequential()
    model.add(layers.Dense(256, activation='relu', input_shape=(input_shape,)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(128, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(2, activation='softmax'))  # 2 output neurons for binary classification (scream or non-scream)

    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    return model

In [10]:
# Evaluate the model
def evaluate_model(model, X_test, y_test):
    loss, accuracy = model.evaluate(X_test, y_test)
    print(f'Test loss: {loss:.4f}')
    print(f'Test accuracy: {accuracy:.4f}')

In [21]:
# Training the model
def train_model(X_train, y_train, X_test, y_test, epochs=150, batch_size=32):
    input_shape = X_train.shape[1]
    model = create_model(input_shape)

    # Callbacks for early stopping and model checkpoint
    # early_stopping = callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

    # Train the model
    history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
                        validation_data=(X_test, y_test))
    # history = model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size,
    #                     validation_data=(X_test, y_test), callbacks=[early_stopping])

    # Plot training & validation accuracy values
    plt.plot(history.history['accuracy'])
    plt.plot(history.history['val_accuracy'])
    plt.title('Model accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    # Plot training & validation loss values
    plt.plot(history.history['loss'])
    plt.plot(history.history['val_loss'])
    plt.title('Model loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(['Train', 'Test'], loc='upper left')
    plt.show()

    return model, history
X_train, X_test, y_train, y_test, label_encoder = load_and_preprocess_data(scream_folder, non_scream_folder)
model, history = train_model(X_train, y_train, X_test, y_test)
evaluate_model(model, X_test, y_test)
# Example to train the model, evaluate, and test with recorded audio
# Update scream_folder to point to the directory containing scream audio files
# scream_folder = "/content/scream_audio_directory"  # Replace with the actual directory path
# non_scream_folder = "/content/clnsp104.wav"

# # Function to load and preprocess the dataset
# def load_and_preprocess_data(scream_folder, non_scream_folder):
#     features = []
#     labels = []

#     # Load scream data
#     import os
#     #Check if scream_folder is a directory, if not directly extract features
#     if os.path.isdir(scream_folder):
#         for filename in os.listdir(scream_folder):
#             path = os.path.join(scream_folder, filename)
#             feature = extract_features(path)
#             features.append(feature)
#             labels.append(1)  # Use integer labels, e.g., 1 for scream
#     else:
#         feature = extract_features(scream_folder)
#         features.append(feature)
#         labels.append(1)  # Use integer labels, e.g., 1 for scream

#     # Load non-scream data
#     #Check if non_scream_folder is a directory, if not directly extract features
#     if os.path.isdir(non_scream_folder):
#         for filename in os.listdir(non_scream_folder):
#             path = os.path.join(non_scream_folder, filename)
#             feature = extract_features(path)
#             features.append(feature)
#             labels.append(0)  # Use integer labels, e.g., 0 for non-scream
#     else:
#         feature = extract_features(non_scream_folder)
#         features.append(feature)
#         labels.append(0)  # Use integer labels, e.g., 0 for non-scream

#     # ... (rest of the function remains the same)

<ipython-input-5-9d25bc728950>:3: UserWarning: PySoundFile failed. Trying audioread instead.
  y, sr = librosa.load(file_path, mono=True)


FileNotFoundError: [Errno 2] No such file or directory: '/content/scream_audio_directory'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [13]:
# Function to predict whether an audio file contains a scream or not
def predict_audio(file_path, model, label_encoder):

    feature = extract_features(file_path)
    feature = np.array(feature).reshape(1, -1)
    prediction = model.predict(feature)
    display(Audio(file_path))

    if len(prediction) == 0:
        return "Unknown"

    predicted_label = label_encoder.inverse_transform([np.argmax(prediction)])
    return predicted_label[0]

In [17]:
# Example to test an audio file

# Test : 1

def train_model(X_train, y_train, X_test, y_test, epochs=150, batch_size=32):

    audio_file_to_test = "/content/343916__reitanna__screaming-and-why.wav"
    predicted_label = predict_audio(audio_file_to_test, model, label_encoder)
    print(f"The predicted label for {audio_file_to_test} is: {predicted_label}")
    print()

    if predicted_label == 1:
        print("Nearby Officer is alerted")
    else:
        print("No alert required")

In [18]:
# Example to test an audio file

# Test : 2

audio_file_to_test = "Assets/testing/p1.wav"
predicted_label = predict_audio(audio_file_to_test, model, label_encoder)
print(f"The predicted label for {audio_file_to_test} is: {predicted_label}")
print()

if predicted_label == 1:
  print("Nearby Officer is alerted")
else:
  print("No alert required")

NameError: name 'model' is not defined

In [ ]:
# Function to record audio and save it to testing
def record_audio(output_file, duration=10, sample_rate=44100):
    print(f"Recording... Please speak for {duration} seconds.")

    # Record audio
    audio_data = sd.rec(int(duration * sample_rate), samplerate=sample_rate, channels=1, dtype=np.int16)
    sd.wait()

    # Save the recorded audio to a file
    write(output_file, sample_rate, audio_data)

    print(f"Recording saved to {output_file}.")

In [ ]:
# Example to record audio and test the prediction
output_file_path = "Assets/testing/recorded_audio.wav"
record_audio(output_file_path)

# Predict the label for the recorded audio
predicted_label = predict_audio(output_file_path, model, label_encoder)
print(f"The predicted label for the recorded audio is: {predicted_label}")
print()

if predicted_label == 1:
    print("Nearby Officer is alerted")
else:
    print("No alert required")

Recording... Please speak for 10 seconds.
Recording saved to Assets/testing/recorded_audio.wav.
1/1 [==============================] - 0s 34ms/step


The predicted label for the recorded audio is: 0

No alert required
